## Vocab Filter Test
partiall
full

17-object, Massive Memory is the correspodning eval dataset in science paper.
64 classes after baby filter.

In [ ]:
import pandas as pd
from utils import load_baby_vocab, vocab_class_filter 

df = pd.read_excel('/home/Dataset/xueyi/KonkLab/17-objects/MM2-Ranks.xls')
class_names = df['Category'].tolist()

vocab_set = set(load_baby_vocab())

filtered_classes = vocab_class_filter(class_names, vocab_set, match_type='full')
print(len(filtered_classes))
print(filtered_classes)

In [ ]:
import importlib
from utils import load_baby_vocab, vocab_class_filter 

class_names = ["siamese cat", "blue+whale", "toy_poodle"]
vocab_set = set(load_baby_vocab())
filtered_classes = vocab_class_filter(class_names, vocab_set, match_type='full')
print(filtered_classes)

## Test Trial Dataset

In [ ]:
import torch
from torchvision import transforms
from datasets.object_dataset import KonkTrialDataset
import matplotlib.pyplot as plt

transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = KonkTrialDataset(trials_file='datasets/trials/object_trials_42.json', transform=transform)

print(f'Dataset length: {len(dataset)}')

sample_idx = 0
imgs, label = dataset[sample_idx]

print(f'Sample {sample_idx} - Label: {label}')
print(f'Sample {sample_idx} - Images shape: {imgs.shape}')

assert isinstance(imgs, torch.Tensor), "Images should be a tensor"
assert imgs.shape == (4, 3, 224, 224), f"Expected image shape (4, 3, 224, 224), but got {imgs.shape}"

combined_imgs = torch.cat([imgs[i] for i in range(imgs.shape[0])], dim=2)

# 因为matplotlib预期图像的格式是 (高度, 宽度, 通道数)，我们需要调整张量的顺序
combined_imgs = combined_imgs.permute(1, 2, 0)  # 重新排列维度为 (224, 896, 3)

# 将图像数据的值归一化到 [0, 1]，因为它们可能已经被标准化处理过
combined_imgs = combined_imgs.numpy()  # 转换为numpy数组
combined_imgs = (combined_imgs - combined_imgs.min()) / (combined_imgs.max() - combined_imgs.min())

# 使用matplotlib显示图像
plt.figure(figsize=(10, 2.5))  # 设置图像显示大小
plt.imshow(combined_imgs)
plt.axis('off')  # 关闭坐标轴
plt.show()

## Trial Predictor Test

In [26]:
from data_utils import get_model, get_dataset

device = 'cuda'
model_name = 'clip'
dataset = 'object-trial'
trials_file = 'datasets/trials/object_img_5_3_42.json'
model, transform = get_model(model_name, device)

data = get_dataset(dataset_name=dataset, transform=transform, trials_file=trials_file,)

sample_idx = 0
imgs, label = data[sample_idx]
print(f'Sample {sample_idx} - Label: {label}')

Loading CLIP...
Successfully loaded CLIP-ViT-L/14
Loading dataset: object-trial
Sample 0 - Label: microwave


In [27]:
from collections import defaultdict
from models.zs_trial_predic import ZSTrialClassifier
 
correct_pred = 0
total_pred = 0
correct_cls_pred = defaultdict(int)
total_cls_predic = defaultdict(int)

classifier = ZSTrialClassifier(model_name, model, device)

with torch.no_grad():
    imgs = imgs.unsqueeze(0).to(device)
    print(label)
    batch_size, per_trial_img_num, channels, height, width = imgs.size()
    imgs = imgs.view(-1, channels, height, width)
    img_features = classifier.get_img_feature(imgs)  # [batch*4, 512]
    img_features = img_features.view(batch_size, per_trial_img_num, -1)  
    img_features = classifier.norm_features(img_features) # [batch, 4, 512]
    txt_features = classifier.get_txt_feature(label)  # [batch, 512]
    txt_features = classifier.norm_features(txt_features) 
    txt_features = txt_features.unsqueeze(1) # [batch, 1, 512]
    
    # Calculate the cosine similarity
    similarity = (100.0 * img_features @ txt_features.transpose(-2, -1)).softmax(dim=1)  # [batch, 4, 1]
    print(f"similarity: {similarity}")
    similarity = similarity.squeeze(-1) # Remove the last dimension
    
    for i in range(batch_size):
        simil = similarity[i]  # Get the similarity scores for the i-th item in the batch
        print(simil)
        predic_idx = simil.argmax().item()  # Find the index of the max similarity score for each trial
        print(predic_idx)
        if predic_idx == 0:  # gt is the first image
            correct_pred += 1
            correct_cls_pred[label[i]] += 1
        total_pred += 1
        total_cls_predic[label[i]] += 1


microwave


similarity: tensor([[[9.9902e-01],
         [6.7186e-04],
         [9.0003e-06],
         [1.5104e-04]]], device='cuda:0', dtype=torch.float16)
tensor([9.9902e-01, 6.7186e-04, 9.0003e-06, 1.5104e-04], device='cuda:0',
       dtype=torch.float16)
0
